This notebook is used to generate the HDF5 datasets `h5_datasets` used in `../train` and `../eval` from the Berry methylation profiling dataset.

In [1]:
from collections import defaultdict
import glob
import os
import random

import h5py
import numpy as np
import pandas as pd

In [2]:
from selene_sdk.samplers import RandomPositionsSampler
from selene_sdk.sequences import Genome
from selene_sdk.targets.genomic_features_h5 import GenomicFeaturesH5

In [3]:
seed = 121
random.seed(seed)
np.random.seed(seed)

In [4]:
SEQ_LEN = 4096

In [5]:
data_dir = './berry_for_ML'

In [6]:
df = pd.read_csv(data_dir + '/global_mapping_ML.bed', sep='\t', header=None)
df.head()

,0,1,2,3
0,chr1,10469,10471,0
1,chr1,10471,10473,1
2,chr1,10484,10486,2
3,chr1,10489,10491,3
4,chr1,10493,10495,4


In [7]:
df.shape

(28564419, 4)

In [8]:
train_coords = df[~df[0].isin(['chr8', 'chr9', 'chr10', 'chrX', 'chrY'])]

In [9]:
train_coords[0].unique()

array(['chr1', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16',
       'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22',
       'chr3', 'chr4', 'chr5', 'chr6', 'chr7'], dtype=object)

In [10]:
valid_coords = df[df[0].isin(['chr10'])]
test_coords = df[df[0].isin(['chr8', 'chr9'])]

In [11]:
train_coords.shape, valid_coords.shape, test_coords.shape

((23285040, 4), (1369946, 4), (2543503, 4))

In [12]:
valid_coords[0].unique(), test_coords[0].unique()

(array(['chr10'], dtype=object), array(['chr8', 'chr9'], dtype=object))

In [13]:
del df

In [14]:
tgts = h5py.File(data_dir + "/global_mapping_tracks.h5", 'r')['targets'][()]

In [ ]:
N_targets = 296
genome = Genome('../resources/hg38_UCSC.fa')
targets = np.arange(N_targets)

target_query = GenomicFeaturesH5(
    data_dir + "/global_mapping_ML.bed.gz",
    data_dir + "/global_mapping_tracks.h5",
    targets
)

sampler = RandomPositionsSampler(
    genome,
    target_query,
    targets,
    seed=seed,
    test_holdout=['chr8', 'chr9'],
    validation_holdout=['chr10'],
    exclude_chrs=['_', 'chrX', 'chrY', 'chrM'],
    sequence_length=SEQ_LEN,
    center_bin_to_predict=[SEQ_LEN // 2,
                           SEQ_LEN // 2 + 1],
    feature_thresholds=None
)

In [16]:
OUTDIR = './h5_datasets'
os.makedirs(OUTDIR, exist_ok=True)
OUTDIR

'./h5_datasets'

We have already randomly sampled subsets from each of `train_coords`, `valid_coords`, and `test_coords` which we load here for reproducing the dataset HDF5 generation.

In [19]:
mode = 'train'
N = 12000000

train0_out = os.path.join(OUTDIR, '{0}.seqlen={1}.seed={2}.N={3}.h5'.format(
                                    mode, SEQ_LEN, seed, N))
print(train0_out)

train_coords0 = np.load(os.path.join(COORDS, 'train.coords.seed=121.N=12000000.tsv.npy'))
train_coords0[:5]

./global.positives_set/train.seqlen=4096.seed=121.N=12000000.h5


array([['chr7', '57479286'],
       ['chr20', '6920231'],
       ['chr2', '11132210'],
       ['chr20', '17805757'],
       ['chr12', '204480']], dtype='<U9')

In [20]:
mode = 'train'
N = 12000000

train1_out = os.path.join(OUTDIR, '{0}.seqlen={1}.seed={2}.N={3}.1.h5'.format(
                                    mode, SEQ_LEN, seed, N))
print(train1_out)
train_coords1 = np.load(os.path.join(COORDS, 'train.coords.seed=121.N=12000000.1.tsv.npy'))
train_coords1[:5]

./global.positives_set/train.seqlen=4096.seed=121.N=12000000.1.h5


array([['chr2', '6276629'],
       ['chr19', '50937945'],
       ['chr19', '21447306'],
       ['chr12', '53341523'],
       ['chr4', '40462695']], dtype='<U9')

In [21]:
mode = 'validate'
N = 32000

valid_out = os.path.join(OUTDIR, '{0}.seqlen={1}.seed={2}.N={3}.h5'.format(
                                    mode, SEQ_LEN, seed, N))
print(valid_out)
valid_coords = np.load(os.path.join(COORDS, 'validate.coords.seed=121.N=32000.tsv.npy'))
valid_coords[:5]

./global.positives_set/validate.seqlen=4096.seed=121.N=32000.h5


array([['chr10', '1731297'],
       ['chr10', '133141284'],
       ['chr10', '114043516'],
       ['chr10', '83803929'],
       ['chr10', '114004460']], dtype='<U9')

In [22]:
mode = 'test'
N = 600000

test_out = os.path.join(OUTDIR, '{0}.seqlen={1}.seed={2}.N={3}.h5'.format(
                                    mode, SEQ_LEN, seed, N))
print(test_out)
test_coords = np.load(os.path.join(COORDS, 'test.coords.seed=121.N=600000.tsv.npy'))
test_coords[:5]

./global.positives_set/test.seqlen=4096.seed=121.N=600000.h5


array([['chr9', '111493894'],
       ['chr9', '115243655'],
       ['chr9', '32958568'],
       ['chr8', '125227457'],
       ['chr8', '133297608']], dtype='<U9')

In [23]:
cg = np.array([[0., 1., 0., 0.],
               [0., 0., 1., 0.]])

In [43]:
def get_positives(out_filename, sampler, pos_coords, N):
    with h5py.File(out_filename, 'a') as fh:
        if 'sequences' in fh:
            del fh['sequences']
            del fh['targets']
            del fh['sequences_length']
        out_fileprefix = out_filename.rsplit('.', 1)[0]
        
        na_count = 0
        sequences = []
        targets = []
        coords = []
        for chrom, pos in pos_coords:
            pos = int(pos)
            output = sampler._retrieve(chrom, pos)
            if output is None:
                na_count += 1
                continue
            seq, tgts = output
            assert np.all(seq[2047:2049] == cg)
            
            seq = np.packbits(seq > 0, axis=0)
            sequences.append(seq)
            targets.append(tgts)
            coords.append((chrom, pos))
            if len(coords) == N:
                break
        
        seqs_mat = np.zeros((len(pos_coords), sequences[0].shape[0], 4))
        for ix, seq in enumerate(sequences):
            seqs_mat[ix] = seq
        
        targets = np.vstack(targets)
        fh.create_dataset('sequences_length', data=SEQ_LEN)
        fh.create_dataset('sequences', data=sequences)
        fh.create_dataset('targets', data=targets)    
        np.save('{0}.coords.tsv'.format(out_fileprefix), coords)
        print(na_count)

In [44]:
get_positives(valid_out, sampler, valid_coords, 32000)

0


In [85]:
get_positives(test_out, sampler, test_coords, 600000)

0


In [ ]:
get_positives(train0_out, sampler, train_coords0, 12000000)

In [ ]:
get_positives(train1_out, sampler, train_coords1, 12000000)